In [36]:
#Import our Libs. I think we might need: 
# os to deal with filepath stuff
# sqlalchemy for ORM and sqqlite operations
# will probably need create_engine, MetaData for reflection into our ORM classes
# pandas for data cleaning

import os
import sqlalchemy
from sqlalchemy import create_engine, MetaData
import pandas as pd
import numpy as np 
# Let's get a little wild. This library uses some interesting techniques but is not sota. statsmodels has a MICE implementation which I'll try later
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# Some additional sqlalchemy things to assist in making a new table and populating it
from sqlalchemy import Table, TEXT, Column, Integer, REAL, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine 
from sqlalchemy.orm import sessionmaker

# Surf's up eh?


In [37]:
# Create an engine for our sqlite database:
hawaii_path = "Homework\homework_08\sqlalchemy_challenge\data\hawaii.sqlite"
# Setting up the engine, connection, and database reflection:
aloha_metadata = MetaData()
hawaii_engine = create_engine(f'sqlite:///{hawaii_path}')
aloha_connect = hawaii_engine.engine.connect()
Session = sessionmaker(bind=hawaii_engine) 
session = Session()

In [38]:
# Now we do the database reflection and check to see if it worked:
aloha_metadata.reflect(bind=hawaii_engine)
aloha_metadata.tables.keys()
#aloha_metadata.tables['measurement']


dict_keys(['imputed_measure', 'measurement', 'station'])

In [39]:
measurement_table = aloha_metadata.tables['measurement']
station_table = aloha_metadata.tables['station']
station_table

Table('station', MetaData(bind=None), Column('id', INTEGER(), table=<station>, primary_key=True, nullable=False), Column('station', TEXT(), table=<station>), Column('name', TEXT(), table=<station>), Column('latitude', FLOAT(), table=<station>), Column('longitude', FLOAT(), table=<station>), Column('elevation', FLOAT(), table=<station>), schema=None)

In [40]:
# sqlite only supports a few alter table functions but luckily ADD Column is on of them. Hopefully this little bit of sql does the trick:
# hawaii_engine.execute("ALTER TABLE measurement ADD COLUMN imp_prcp real")
#Ok after reading about it for a little bit sqlite doesn't really support if not exist for alter table. it makes more sense to create a table with just the imputed values and create a view to querry. That seems to be the most 'sql' way.

#let's make our table class for our orm:
# it seems like the wrong way to do it, but this table will have the same schema as the measurement tabe. I should just reflect the table onto another one and populate it. This way I get some practice specifying the tables.

# I actually just copied and pasted the output of third cell. It's nearly complete as it is

base = declarative_base()
class Imp_measure(base):
    __tablename__  = 'imputed_measure'

    id = Column(Integer, primary_key = True)
    station = Column('station', TEXT(), ForeignKey('measurement.station'))
    date = Column('date', TEXT(), ForeignKey('measurement.date'))
    imp_prcp = Column('prcp', REAL())
    tobs = Column('tobs', REAL(), ForeignKey('measurement.tobs'))

    def __repr__(self):
        return f'station={self.station}\
                date={self.date}\
                imp_prcp{self.imp_prcp}\
                tobs={self.tobs}'

In [41]:
base.metadata.create_all(hawaii_engine)
imputed_table = aloha_metadata.tables['imputed_measure']
imputed_table

Table('imputed_measure', MetaData(bind=None), Column('id', BIGINT(), table=<imputed_measure>), Column('station', TEXT(), table=<imputed_measure>), Column('date', TEXT(), table=<imputed_measure>), Column('prcp', FLOAT(), table=<imputed_measure>), Column('tobs', FLOAT(), table=<imputed_measure>), schema=None)

In [42]:
# Now that we can call the tables lets run our querry from before to see if we can get our null values and then we'll see about imputation or extra data retrieval

len(hawaii_engine.execute("SELECT * FROM measurement WHERE prcp ISNULL ").fetchall())


1447

In [43]:
# Check and see if we can call the ORM object we reflected when we reflected the database. 
measurement_table.columns.prcp

Column('prcp', FLOAT(), table=<measurement>)

In [44]:
#
measurement_df = pd.read_sql_table('measurement', aloha_connect)
measurement_df['prcp']

0        0.08
1        0.00
2        0.00
3        0.00
4         NaN
         ... 
19545    0.09
19546     NaN
19547    0.56
19548    0.50
19549    0.45
Name: prcp, Length: 19550, dtype: float64

In [45]:
# Gonna quick try out something I found in a book which use scikit-learn and it's imputation function to avoid filling the null's with the mean(This will distort your data) or discarding the row entry all together. Ultimatley which technique is most efficacious depends on the nature of the missingness of the value. The rows have other information that we do not want to discard. I have been reading about missingness and it is a fascinating subject. See: https://us.sagepub.com/en-us/nam/best-practices-in-data-cleaning/book235006 I'm just playing around, I'm not 100% sure about what it is doing.
print(measurement_df.shape)

fit_impute_df = measurement_df.sample(8000)
test_impute_df = measurement_df.sample(10000)
imp = IterativeImputer(max_iter=10000, random_state=0)
imp.fit(fit_impute_df[['prcp','tobs']])
print(np.round(imp.transform(test_impute_df[['prcp','tobs']]),2))
test_impute_df
# Ok that's pretty neat. How lovely that it takes pandas dataframes as args Let's see how that works on the whole data frame:
test_full_measure_df = measurement_df
full_impute_df = pd.DataFrame(np.round(imp.transform(test_full_measure_df[['prcp', 'tobs']]),2), columns = ['prcp', 'tobs'])
test_full_measure_df[['prcp_imp', 'tobs_imp']] = full_impute_df[['prcp', 'tobs']]
test_full_measure_df
#Bingo!
# I did some more reading. skl is actually using the np.array that the pandas dataframe is built upon. That is why it returns and array and it has to stuffed back into a data frame.

(19550, 5)
[[ 0. 74.]
 [ 0. 78.]
 [ 0. 71.]
 ...
 [ 0. 73.]
 [ 0. 73.]
 [ 0. 80.]]


,id,station,date,prcp,tobs,prcp_imp,tobs_imp
0,1,USC00519397,2010-01-01,0.08,65.0,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0,0.16,73.0
...,...,...,...,...,...,...,...
19545,19546,USC00516128,2017-08-19,0.09,71.0,0.09,71.0
19546,19547,USC00516128,2017-08-20,NaN,78.0,0.10,78.0
19547,19548,USC00516128,2017-08-21,0.56,76.0,0.56,76.0
19548,19549,USC00516128,2017-08-22,0.50,76.0,0.50,76.0


In [46]:
clean_measure_df = test_full_measure_df[['id','station','date', 'prcp_imp', 'tobs_imp']]
clean_measure_df.head()

,id,station,date,prcp_imp,tobs_imp
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,0.16,73.0


In [47]:
clean_columns = ['id', 'station', 'date', 'prcp', 'tobs']
clean_measure_df.columns = clean_columns
clean_measure_df.head(10)

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,0.16,73.0
5,6,USC00519397,2010-01-07,0.06,70.0
6,7,USC00519397,2010-01-08,0.00,64.0
7,8,USC00519397,2010-01-09,0.00,68.0
8,9,USC00519397,2010-01-10,0.00,73.0
9,10,USC00519397,2010-01-11,0.01,64.0


In [55]:
imputed_table

Table('imputed_measure', MetaData(bind=None), Column('id', BIGINT(), table=<imputed_measure>), Column('station', TEXT(), table=<imputed_measure>), Column('date', TEXT(), table=<imputed_measure>), Column('prcp', FLOAT(), table=<imputed_measure>), Column('tobs', FLOAT(), table=<imputed_measure>), schema=None)

In [56]:
clean_measure_df.to_sql('imputed_measureimputed_tabl, hawaii_engine, index=False, if_exists='replace')

AttributeError: 'Table' object has no attribute 'replace'